In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

# run

## initialize log

In [ ]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [ ]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
  Time Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0  189  0  0    366    347   348  0.09  0.52  0.65  0.48  0.30
1  232  1  0    328    320   323  0.10  0.53  0.68  0.45  0.26
2  272  2  0    316    313   313  0.10  0.52  0.69  0.44  0.27
3  276  3  0    314    313   313  0.10  0.52  0.68  0.36  0.23
4  298  4  0    314    315   313  0.10  0.54  0.66  0.39  0.28
5  324  5  0    307    310   308  0.10  0.56  0.65  0.36  0.27
6  340  6  0    300    300   299  0.11  0.56  0.64  0.35  0.26
7  374  7  0    298    302   299  0.11  0.56  0.64  0.36  0.26
8  377  8  0    294    295   294  0.12  0.56  0.63  0.35  0.26

 0 31943 279164.0 ! nice price bought love quality made perfect recommend 'm
   0-1 1746 13162.0 ! love color cover mac pro perfectly apple purple keyboard
     0-1-1 6 11.0 memory brown smooth pretty friend dropping padded compact ugly flash
     0-1-2 230 196.0 cable deal sleeve company shell compliments hands drive remove leaving
     0-1-3 2 7.0 adapter ports thought clean lining video zips fitting fingerprints finish
     0-1-4 307 202.0 durability solid worried cleaning clean shipping people person matte colors
     0-1-5 1201 1552.0 keyboard cover protector mcover typing pink key keys seller highly
   0-2 929 8257.0 carry travel work room pockets stuff plenty compartments back trip
     0-2-1 197 215.0 ground organization extras important lots files worn choose car fully
     0-2-2 3 8.0 waterproof papers convenient recommended breaking base space start fingerprints finish
     0-2-3 729 808.0 camera equipment gear lens lenses comfortable lot cards canon easy
   0-3 1756 12258.0